In [34]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from utils import ingresos_financieros, meses_entre_fechas
from impactos_iniciativa import ImpactosIniciativa

# Funciones internas útiles

In [35]:
# función para calcular el precio dada una tabla de precios externa de excel
def aplicar_precio(monto: float, riesgo: str, precios: pd.DataFrame) -> float:
    if monto <= 500000:
        precio = precios[precios["Rangos"] >= monto][riesgo].reset_index(drop=True)[0]
    elif monto > 500000:
        precio = None
    return precio

In [36]:
creditos = pd.read_csv(r"C:\Users\mariajose_chinchilla\Downloads\info_creditos_cancelados.csv", sep=";")
creditos["PORCENTAJE SALDO"] = 1 - creditos["SALDO"] / creditos["MONTO_DESEMBOLSADO"]
creditos["FECHA_APERTURA_CR_CANCELADO"] = pd.to_datetime(creditos["FECHA_APERTURA_CR_CANCELADO"])

In [ ]:
analizar = ImpactosIniciativa(creditos, "MONTO_DESEMBOLSADO", 
                              "PLAZO", "TASA", "FECHA_APERTURA_CR_CANCELADO", "DESEMBOLSO_NETO")

# Empezar análisis de impactos

## Créditos cancelados por recrédito

In [ ]:
analizar.flag15_total("CREDITO", "PORCENTAJE SALDO")
analizar.ingresos_recibidos("YA RECIBIDO", False)
analizar.margen_financiero("YA RECIBIDO", False)

analizar.ingresos_recibidos("CANCELADOS POR RECREDITO", True)
analizar.margen_financiero("CANCELADOS POR RECREDITO", False)

## Recréditos desembolsados

In [52]:
# Separar por saldo interno para procesar los datos
desembolsos = pd.read_csv(r"C:\Users\mariajose_chinchilla\Downloads\informacion_desembolsos_v2.csv", 
                          sep=";", encoding='cp1252' )

#df_flag = analizar.creditos[["CREDITO", "PAGA RESERVAS"]]
#desembolsos = pd.merge(desembolsos, df_flag, how="left", on="CREDITO")

In [42]:
# leer archivo separadamente segun precios
precios_sf80 = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\FocoBanruralPrecios.xlsx", sheet_name="SF80")
precios_sf80_120 = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\FocoBanruralPrecios.xlsx", sheet_name="SF80-120")
precios_sf120 = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\FocoBanruralPrecios.xlsx", sheet_name="SF120")

precios_liq80 = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\FocoBanruralPrecios.xlsx", sheet_name="L80")
precios_liq80_120 = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\FocoBanruralPrecios.xlsx", sheet_name="L80-120")
precios_liq120 = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\FocoBanruralPrecios.xlsx", sheet_name="L120")

In [ ]:
# usar mascaras para procesar diferentes los precios
mascara_liquidez = desembolsos["PRODUCTO"].isin(["SOLUCION LIQUIDEZ CON FIADOR",'SOLUCION LIQUIDEZ SIN FIADOR'])
mascara_menor80 = desembolsos["SALDO_BT"] <= 80000
mascara80_120 = (desembolsos["SALDO_BT"] > 80000) & (desembolsos["SALDO_BT"] <= 120000)

In [ ]:
# Definir conjuntos de datos
sf80 = desembolsos.loc[~mascara_liquidez & mascara_menor80]
sf80_120 = desembolsos.loc[~mascara_liquidez & mascara80_120]
sf_120 = desembolsos.loc[~mascara_liquidez & ~(mascara_menor80 & mascara80_120)]

liq80 = desembolsos.loc[mascara_liquidez & mascara_menor80]
liq80_120 = desembolsos.loc[mascara_liquidez & mascara80_120]
liq_120 = desembolsos.loc[mascara_liquidez & ~(mascara_menor80 & mascara80_120)]

In [ ]:
# # instanciar las clases
# validar_sf80 = ImpactosIniciativa(sf80, "MONTO_DESEMBOLSADO", "PLAZO", "TASA", "FECHA_APERTURA", "DESEMBOLSO_NETO", precios_sf80)
# validar_sf80_120 = ImpactosIniciativa(sf80_120, "MONTO_DESEMBOLSADO", "PLAZO", "TASA", "FECHA_APERTURA", "DESEMBOLSO_NETO", precios_sf80_120)
# validar_sf120 = ImpactosIniciativa(sf_120, "MONTO_DESEMBOLSADO", "PLAZO", "TASA", "FECHA_APERTURA", "DESEMBOLSO_NETO", precios_sf120)

# validar_liq80 = ImpactosIniciativa(liq80, "MONTO_DESEMBOLSADO", "PLAZO", "TASA", "FECHA_APERTURA", "DESEMBOLSO_NETO", precios_liq80)
# validar_liq80_120 = ImpactosIniciativa(liq80_120, "MONTO_DESEMBOLSADO", "PLAZO", "TASA", "FECHA_APERTURA", "DESEMBOLSO_NETO", precios_sf80_120)
# validar_liq120 = ImpactosIniciativa(liq_120, "MONTO_DESEMBOLSADO", "PLAZO", "TASA", "FECHA_APERTURA", "DESEMBOLSO_NETO", precios_liq120)

In [ ]:
# Pegar precio
sf80["PRECIO NUEVO"] = sf80.apply(lambda x: aplicar_precio(x["MONTO_DESEMBOLSADO"], x["score"], precios_sf80), axis=1)
sf80_120["PRECIO NUEVO"] = sf80_120.apply(lambda x: aplicar_precio(x["MONTO_DESEMBOLSADO"], x["score"], precios_sf80_120), axis=1)
sf_120["PRECIO NUEVO"] = sf_120.apply(lambda x: aplicar_precio(x["MONTO_DESEMBOLSADO"], x["score"], precios_sf80_120), axis=1)

liq80["PRECIO NUEVO"] = liq80.apply(lambda x: aplicar_precio(x["MONTO_DESEMBOLSADO"], x["score"], precios_liq80), axis=1)
liq80_120["PRECIO NUEVO"] = liq80_120.apply(lambda x: aplicar_precio(x["MONTO_DESEMBOLSADO"], x["score"], precios_liq80_120), axis=1)
liq_120["PRECIO NUEVO"] = liq_120.apply(lambda x: aplicar_precio(x["MONTO_DESEMBOLSADO"], x["score"], precios_liq120), axis=1)

In [ ]:
# Procesar datos
sf80["INGRESOS A 2024"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"]/30, 6), axis=1)
sf80_120["INGRESOS A 2024"] = sf80_120.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"]/30, 6), axis=1)
sf_120["INGRESOS A 2024"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"]/30, 6), axis=1)


liq80["INGRESOS A 2024"] = liq80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"]/30, 6), axis=1)
liq80_120["INGRESOS A 2024"] = liq80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"]/30, 6), axis=1)
liq_120["INGRESOS A 2024"] = liq80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["TASA"]/1200, x["PLAZO"]/30, 6), axis=1)

In [ ]:
sf80["RESERVAS REESTRUCTURACION"] = 0.45 * 0.05 * sf80["MONTO_DESEMBOLSADO"]
sf80_120["RESERVAS REESTRUCTURACION"] = 0.45 * 0.05 * sf80["MONTO_DESEMBOLSADO"]
sf_120["RESERVAS REESTRUCTURACION"] = 0.45 * 0.05 * sf80["MONTO_DESEMBOLSADO"]

liq80["RESERVAS REESTRUCTURACION"] = 0.45 * 0.05 * sf80["MONTO_DESEMBOLSADO"]
liq80_120["RESERVAS REESTRUCTURACION"] = 0.45 * 0.05 * sf80["MONTO_DESEMBOLSADO"]
liq_120["RESERVAS REESTRUCTURACION"] = 0.45 * 0.05 * sf80["MONTO_DESEMBOLSADO"]

In [ ]:
# margen financiero
sf80["MARGEN FINANCIERO NUEVO"] = sf80["INGRESOS A 2024"] - sf80["RESERVAS REESTRUCTURACION"] - 0.0747/2 * sf80["DESEMBOLSO_NETO"]
sf80_120["MARGEN FINANCIERO NUEVO"] = sf80_120["INGRESOS A 2024"] - sf80_120["RESERVAS REESTRUCTURACION"] - 0.0747/2 * sf80_120["DESEMBOLSO_NETO"]
sf_120["MARGEN FINANCIERO NUEVO"] = sf_120["INGRESOS A 2024"] - sf_120["RESERVAS REESTRUCTURACION"] - 0.0747/2 * sf_120["DESEMBOLSO_NETO"]

liq80["MARGEN FINANCIERO NUEVO"] = liq80["INGRESOS A 2024"] - liq80["RESERVAS REESTRUCTURACION"] - 0.0747/2 * liq80["DESEMBOLSO_NETO"]
liq80_120["MARGEN FINANCIERO NUEVO"] = liq80_120["INGRESOS A 2024"] - liq80_120["RESERVAS REESTRUCTURACION"] - 0.0747/2 * liq80_120["DESEMBOLSO_NETO"]
liq_120["MARGEN FINANCIERO NUEVO"] = liq_120["INGRESOS A 2024"] - liq_120["RESERVAS REESTRUCTURACION"] - 0.0747/2 * liq_120["DESEMBOLSO_NETO"]

In [ ]:
# margen financiero CON NUESTRAS TASAS
# Procesar datos
sf80["INGRESOS A 2024 FOCO BANRURAL"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["PRECIO NUEVO"]/1200, x["PLAZO"]/30, 6), axis=1)
sf80_120["INGRESOS A 2024 FOCO BANRURAL"] = sf80_120.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["PRECIO NUEVO"]/1200, x["PLAZO"]/30, 6), axis=1)
sf_120["INGRESOS A 2024 FOCO BANRURAL"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["PRECIO NUEVO"]/1200, x["PLAZO"]/30, 6), axis=1)


liq80["INGRESOS A 2024 FOCO BANRURAL"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["PRECIO NUEVO"]/1200, x["PLAZO"]/30, 6), axis=1)
liq80_120["INGRESOS A 2024 FOCO BANRURAL"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["PRECIO NUEVO"]/1200, x["PLAZO"]/30, 6), axis=1)
liq_120["INGRESOS A 2024 FOCO BANRURAL"] = sf80.apply(lambda x: ingresos_financieros(x["MONTO_DESEMBOLSADO"], x["PRECIO NUEVO"]/1200, x["PLAZO"]/30, 6), axis=1)

################
sf80["MARGEN FINANCIERO FOCO BANRURAL"] = sf80["INGRESOS A 2024 FOCO BANRURAL"] - 0.0747 * sf80["DESEMBOLSO_NETO"]
sf80_120["MARGEN FINANCIERO FOCO BANRURAL"] = sf80_120["INGRESOS A 2024 FOCO BANRURAL"] - 0.0747 * sf80_120["DESEMBOLSO_NETO"]
sf_120["MARGEN FINANCIERO FOCO BANRURAL"] = sf_120["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747 * sf_120["DESEMBOLSO_NETO"]

liq80["MARGEN FINANCIERO FOCO BANRURAL"] = liq80["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747 * liq80["DESEMBOLSO_NETO"]
liq80_120["MARGEN FINANCIERO FOCO BANRURAL"] = liq80_120["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747 * liq80_120["DESEMBOLSO_NETO"]
liq_120["MARGEN FINANCIERO FOCO BANRURAL"] = liq_120["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747 * liq_120["DESEMBOLSO_NETO"]

In [ ]:
# identificador de datos
sf80["IDENTIFICADOR"] = "SF80"
sf80_120["IDENTIFICADOR"] = "SF80-120"
sf_120["IDENTIFICADOR"] = "SF120"

sf80_120["IDENTIFICADOR"] = "LIQ80"
liq80_120["IDENTIFICADOR"] = "LIQ80-120"
liq_120["IDENTIFICADOR"] = "LIQ120"

## Precalificaciones verdes

In [3]:
verdes_no_desem = pd.read_csv(r"C:\Users\mariajose_chinchilla\Downloads\verdes no desembolsadas 1.csv", sep=";")
verde_cancela = pd.read_csv(r"C:\Users\mariajose_chinchilla\Downloads\creditos a cancelar.csv", sep=";", encoding='cp1252')

In [4]:
verde_cancela["fConsecion"] = pd.to_datetime(verde_cancela["fConsecion"])
verde_cancela["fVencimiento"] = pd.to_datetime(verde_cancela["fVencimiento"])
verde_cancela["PLAZO"] = verde_cancela.apply(lambda x: meses_entre_fechas(x["fConsecion"], x["fVencimiento"]), axis=1)
verde_cancela["PORCENTAJE SALDO"] = 1 - verde_cancela["saldo"]/verde_cancela["capitalOriginal"]

# Ver ingresos y márgenes ya recibidos

In [5]:
analizar_precas = ImpactosIniciativa(verde_cancela, "capitalOriginal", "PLAZO", 
                                     "tasa", "fConsecion", "capitalOriginal")
analizar_precas.flag15_total("no_precalificacion", "PORCENTAJE SALDO")
analizar_precas.ingresos_recibidos("YA RECIBIDO", False)
analizar_precas.margen_financiero("YA RECIBIDO", False)

analizar_precas.ingresos_recibidos("QUE SERÍAN CANCELADOS POR RECREDITO", True)
analizar_precas.margen_financiero("QUE SERÍAN CANCELADOS POR RECREDITO", False)

In [6]:
analizar_precas.creditos.to_csv(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\espejo\Créditos que se cancelarían con las precas.csv")

## Ahora las precas

In [8]:
# pegar si paga o no reservas
df_temp = analizar_precas.creditos[["no_precalificacion", "PAGA RESERVAS"]]
verdes_no_desem = verdes_no_desem[(verdes_no_desem["tipo_producto"] == "SALUD FINANCIERA") | (verdes_no_desem["tipo_producto"] == "LIQUIDEZ")]
verdes_no_desem = pd.merge(verdes_no_desem, df_temp, right_on="no_precalificacion", left_on="Precalificacion",how="left")

In [12]:
# hacer mascaras y definir conjuntos de datos por rango de saldo interno
# usar mascaras para procesar diferentes los precios
mascara_liquidez = verdes_no_desem["tipo_producto"] == "LIQUIDEZ"
mascara_menor80 = verdes_no_desem["saldo_internoBT"] <= 80000
mascara80_120 = (verdes_no_desem["saldo_internoBT"] > 80000) & (verdes_no_desem["saldo_internoBT"] <= 120000)

# definir conjunto de datos
preca_sf80 = verdes_no_desem.loc[~mascara_liquidez & mascara_menor80]
preca_sf_80_120 = verdes_no_desem.loc[~mascara_liquidez & mascara80_120]
preca_sf_120 = verdes_no_desem.loc[~mascara_liquidez & ~(mascara_menor80 & mascara80_120)]

preca_liq80 = verdes_no_desem.loc[mascara_liquidez & mascara_menor80]
preca_liq80_120 = verdes_no_desem.loc[mascara_liquidez & mascara80_120]
preca_liq_120 = verdes_no_desem.loc[mascara_liquidez & ~(mascara80_120 & mascara_menor80)]

In [13]:
# Pegar precio
try:
    preca_sf80["PRECIO NUEVO"] = preca_sf80.apply(lambda x: aplicar_precio(x["solicitudMonto"], x["scoreDeudorInstallmentQualPacing"], precios_sf80), axis=1)
except:
    pass
try:
    preca_sf_80_120["PRECIO NUEVO"] = preca_sf_80_120.apply(lambda x: aplicar_precio(x["solicitudMonto"], x["scoreDeudorInstallmentQualPacing"], precios_sf80_120), axis=1)
except:
    pass
try:
    preca_sf_120["PRECIO NUEVO"] = preca_sf_120.apply(lambda x: aplicar_precio(x["solicitudMonto"], x["scoreDeudorInstallmentQualPacing"], precios_sf80_120), axis=1)
except:
    pass


try:
    preca_liq80["PRECIO NUEVO"] = preca_liq80.apply(lambda x: aplicar_precio(x["solicitudMonto"], x["scoreDeudorInstallmentQualPacing"], precios_liq80), axis=1)
except:
    pass
try:
    preca_liq80_120["PRECIO NUEVO"] = preca_liq80_120.apply(lambda x: aplicar_precio(x["solicitudMonto"], x["scoreDeudorInstallmentQualPacing"], precios_liq80_120), axis=1)
except:
    pass
try:
    preca_liq_120["PRECIO NUEVO"] = preca_liq_120.apply(lambda x: aplicar_precio(x["solicitudMonto"], x["scoreDeudorInstallmentQualPacing"], precios_liq120), axis=1)
except:
    pass

In [14]:
# Procesar datos
preca_sf80["INGRESOS QUE SE TENDRIAN A 2024"] = preca_sf80.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["tasa"]/1200, x["solicitudPlazo"], 6), axis=1)
preca_sf_80_120["INGRESOS QUE SE TENDRIAN A 2024"] = preca_sf_80_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["tasa"]/1200, x["solicitudPlazo"], 6), axis=1)
preca_sf_120["INGRESOS QUE SE TENDRIAN A 2024"] = preca_sf80.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["tasa"]/1200, x["solicitudPlazo"], 6), axis=1)


preca_liq80["INGRESOS QUE SE TENDRIAN A 2024"] = preca_liq80.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["tasa"]/1200, x["solicitudPlazo"], 6), axis=1)
preca_liq80_120["INGRESOS QUE SE TENDRIAN A 2024"] = preca_liq80_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["tasa"]/1200, x["solicitudPlazo"], 6), axis=1)
preca_liq_120["INGRESOS QUE SE TENDRIAN A 2024"] = preca_liq_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["tasa"]/1200, x["solicitudPlazo"], 6), axis=1)

In [15]:
# calcular reservas
preca_sf80.loc[preca_sf80["PAGA RESERVAS"] == 1, "RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] = 0.05 * 0.45 * preca_sf80["solicitudMonto"]
preca_sf_80_120.loc[preca_sf_80_120["PAGA RESERVAS"] == 1, "RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] = 0.05 * 0.45 * preca_sf_80_120["solicitudMonto"]
preca_sf_120.loc[preca_sf_120["PAGA RESERVAS"] == 1, "RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] = 0.05 * 0.45 * preca_sf_120["solicitudMonto"]

preca_liq80.loc[preca_liq80["PAGA RESERVAS"] == 1, "RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] = 0.05 * 0.45 * preca_liq80["solicitudMonto"]
preca_liq80_120.loc[preca_liq80_120["PAGA RESERVAS"] == 1, "RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] = 0.05 * 0.45 * preca_liq80_120["solicitudMonto"]
preca_liq_120.loc[preca_liq_120["PAGA RESERVAS"] == 1, "RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] = 0.05 * 0.45 * preca_liq_120["solicitudMonto"]


# llenar vacios
preca_sf80["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"].fillna(0, inplace=True)
preca_liq80_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"].fillna(0, inplace=True)
preca_liq_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"].fillna(0, inplace=True)

preca_liq80["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"].fillna(0, inplace=True)
preca_liq80_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"].fillna(0, inplace=True)
preca_liq_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"].fillna(0, inplace=True)

In [16]:
# calcular margen financiero con la tasa de la precalificación
preca_sf80["MARGEN FINANCIERO QUE SE TENDRIA"] = preca_sf80["INGRESOS QUE SE TENDRIAN A 2024"] - preca_sf80["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] - (0.0747 / 2) * preca_sf80["solicitudMonto"]
preca_sf_80_120["MARGEN FINANCIERO QUE SE TENDRIA"] = preca_sf_80_120["INGRESOS QUE SE TENDRIAN A 2024"] - preca_sf_80_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] - (0.0747 / 2) * preca_sf_80_120["solicitudMonto"]
preca_sf_120["MARGEN FINANCIERO QUE SE TENDRIA"] = preca_sf_120["INGRESOS QUE SE TENDRIAN A 2024"] - preca_sf_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] - (0.0747 / 2) * preca_sf_120["solicitudMonto"]

preca_liq80["MARGEN FINANCIERO QUE SE TENDRIA"] = preca_liq80["INGRESOS QUE SE TENDRIAN A 2024"] - preca_liq80["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] - (0.0747 / 2) * preca_liq80["solicitudMonto"]
preca_liq80_120["MARGEN FINANCIERO QUE SE TENDRIA"] = preca_liq80_120["INGRESOS QUE SE TENDRIAN A 2024"] - preca_liq80_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] - (0.0747 / 2) * preca_liq80_120["solicitudMonto"]
preca_liq_120["MARGEN FINANCIERO QUE SE TENDRIA"] = preca_liq_120["INGRESOS QUE SE TENDRIAN A 2024"] - preca_liq_120["RESERVAS POR REESTRUCTURACION QUE SE PAGARIAN"] - (0.0747 / 2) * preca_liq_120["solicitudMonto"]

In [29]:
# ingresos y margen con foco banrural
# margen financiero CON NUESTRAS TASAS
# Procesar datos
try:
    preca_sf80["INGRESOS A 2024 FOCO BANRURAL"] = preca_sf80.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["PRECIO NUEVO"]/1200, x["solicitudPlazo"], 6), axis=1)
except:
    pass
try:
    preca_sf_80_120["INGRESOS A 2024 FOCO BANRURAL"] = preca_sf_80_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["PRECIO NUEVO"]/1200, x["solicitudPlazo"]/30, 6), axis=1)
except:
    pass
try:
    preca_sf_120["INGRESOS A 2024 FOCO BANRURAL"] = preca_sf_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["PRECIO NUEVO"]/1200, x["solicitudPlazo"], 6), axis=1)
except:
    pass

try:
    preca_liq80["INGRESOS A 2024 FOCO BANRURAL"] = preca_liq80.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["PRECIO NUEVO"]/1200, x["solicitudPlazo"], 6), axis=1)
    preca_liq80_120["INGRESOS A 2024 FOCO BANRURAL"] = preca_liq80_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["PRECIO NUEVO"]/1200, x["solicitudPlazo"], 6), axis=1)
    preca_liq_120["INGRESOS A 2024 FOCO BANRURAL"] = preca_liq_120.apply(lambda x: ingresos_financieros(x["solicitudMonto"], x["PRECIO NUEVO"]/1200, x["solicitudPlazo"], 6), axis=1)
except:
    pass
################
try:
    preca_sf80["MARGEN FINANCIERO FOCO BANRURAL"] = preca_sf80["INGRESOS A 2024 FOCO BANRURAL"] - 0.0747/2 * preca_sf80["solicitudMonto"]
except:
    pass
try:
    preca_sf_80_120["MARGEN FINANCIERO FOCO BANRURAL"] = preca_sf_80_120["INGRESOS A 2024 FOCO BANRURAL"] - 0.0747/2 * preca_sf_80_120["solicitudMonto"]
except:
    pass
try:
    preca_sf_120["MARGEN FINANCIERO FOCO BANRURAL"] = preca_sf_120["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747/2 * preca_sf_120["solicitudMonto"]
except:
    pass

try:
    preca_liq80["MARGEN FINANCIERO FOCO BANRURAL"] = preca_liq80["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747/2 * preca_liq80["solicitudMonto"]
    preca_liq80_120["MARGEN FINANCIERO FOCO BANRURAL"] = preca_liq80_120["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747/2 * preca_liq80_120["solicitudMonto"]
    preca_liq_120["MARGEN FINANCIERO FOCO BANRURAL"] = preca_liq_120["INGRESOS A 2024 FOCO BANRURAL"]  - 0.0747/2 * preca_liq_120["solicitudMonto"]
except Exception as e:
    print(e)

'INGRESOS A 2024 FOCO BANRURAL'


In [30]:
preca_sf80["IDENTIFICADOR"] = "SF80"
preca_sf_80_120["IDENTIFICADOR"] = "SF80-120"
preca_sf_120["IDENTIFICADOR"] = "SF120"
preca_liq80["IDENTIFICADOR"] = "LIQ80"
preca_liq80_120["IDENTIFICADOR"] = "LIQ80-120"
preca_liq_120["IDENTIFICADOR"] = "LIQ120"

In [31]:
df_total = [preca_sf80, preca_sf_80_120, preca_sf_120, preca_liq80, preca_liq80_120, preca_liq_120]
df_total = pd.concat(df_total)

In [1]:
from utils import ingresos_financieros

In [64]:
cancelado = ingresos_financieros(6200, 23/1200, 60, 11) - ingresos_financieros(6200, 23/1200, 60, 6)
nuevo = ingresos_financieros(15000, 23/1200, 60, 35) - ingresos_financieros(15000, 23/1200, 60, 30)

In [66]:
ingresos_financieros(6200, (0.0747+0.0025)/(12*(1-0.146)), 60, 11) - ingresos_financieros(6200, (0.0747+0.0025)/(12*(1-0.146)), 60, 6)

208.09786156330296